**Importing of Relevant Libraries**

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


**Data Preparation**

In [2]:
#Loading of data
data_raw = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data_val  = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
#Separating Samples and Labels
label = data_raw['label'].copy()
data_raw = data_raw.drop('label', axis=1)

In [4]:
label = label.values
data_raw = data_raw.values

In [5]:
#Creating Training and Validation sets
X_train, X_val, y_train, y_val = train_test_split(data_raw, label, test_size=0.1, random_state=28, shuffle=True, stratify=label)

In [6]:
#Reshaping the pixels to form the Image 
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_val = X_val.reshape(-1, 28, 28, 1) / 255.0

In [7]:
def preprocess(image, label):
    rgb_image = tf.image.grayscale_to_rgb(tf.convert_to_tensor(image)) #Converting the Images to RGB from Grayscale
    resized_image = tf.image.resize(rgb_image, (224, 224)) #Resizing the Images for use in Resnet50
    final_image = keras.applications.resnet50.preprocess_input(resized_image)
    return final_image, label

In [9]:
batch_size = 32

In [10]:
#Splitting the data into batches
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train))
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
train_dataset = train_dataset.map(preprocess, num_parallel_calls=1)

In [11]:
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=1)
val_dataset = val_dataset.batch(batch_size, drop_remainder=True)

**Creating a model with Resnet50 as the Body**

In [12]:
base_model = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dense(256, activation='relu')(avg)
output = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=base_model.input, outputs=output)

94765736/94765736 [==============================] - 0s 0us/step


In [13]:
#Freezing the layers in Resnet50
for layer in base_model.layers:
    layer.trainable = False

In [14]:
#Compiling the model with a Adam optimizer with a relatively higher Learning Rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
 metrics=["accuracy"])

In [23]:
#Creating Callbacks 
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath='model_weights_epoch{epoch:02d}.h5', 
    save_weights_only=True,  
    save_freq='epoch')
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [24]:
epochs = 20

In [25]:
#Training the Head of the model
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[lr_scheduler,early_stopping_cb,checkpoint_cb])

Epoch 1/20
1181/1181 [==============================] - 183s 144ms/step - loss: 0.9960 - accuracy: 0.7113 - val_loss: 0.4610 - val_accuracy: 0.8779 - lr: 0.0010
Epoch 2/20
1181/1181 [==============================] - 160s 135ms/step - loss: 0.3893 - accuracy: 0.8870 - val_loss: 0.2753 - val_accuracy: 0.9229 - lr: 0.0010
Epoch 3/20
1181/1181 [==============================] - 161s 137ms/step - loss: 0.2936 - accuracy: 0.9098 - val_loss: 0.2688 - val_accuracy: 0.9191 - lr: 0.0010
Epoch 4/20
1181/1181 [==============================] - 171s 144ms/step - loss: 0.2531 - accuracy: 0.9201 - val_loss: 0.2534 - val_accuracy: 0.9239 - lr: 0.0010
Epoch 5/20
1181/1181 [==============================] - 169s 143ms/step - loss: 0.2265 - accuracy: 0.9289 - val_loss: 0.1879 - val_accuracy: 0.9451 - lr: 0.0010
Epoch 6/20
1181/1181 [==============================] - 170s 144ms/step - loss: 0.2101 - accuracy: 0.9329 - val_loss: 0.2028 - val_accuracy: 0.9318 - lr: 0.0010
Epoch 7/20
1181/1181 [============

In [42]:
#Unfreezing last layer of base_model 
base_model.layers[-1].trainable = True

In [44]:
#Compiling the model with a Adam optimizer with a relatively lower Learning Rate
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
metrics=["accuracy"])

In [45]:
stage2_checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath='stage2_model_weights_epoch{epoch:02d}.h5', 
    save_weights_only=True,  
    save_freq='epoch')

In [47]:
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[lr_scheduler,early_stopping_cb,stage2_checkpoint_cb])

Epoch 1/20
1181/1181 [==============================] - 177s 146ms/step - loss: 0.1002 - accuracy: 0.9688 - val_loss: 0.1049 - val_accuracy: 0.9668 - lr: 1.0000e-04
Epoch 2/20
1181/1181 [==============================] - 163s 138ms/step - loss: 0.0992 - accuracy: 0.9684 - val_loss: 0.1142 - val_accuracy: 0.9676 - lr: 1.0000e-04
Epoch 3/20
1181/1181 [==============================] - 176s 149ms/step - loss: 0.0988 - accuracy: 0.9688 - val_loss: 0.1116 - val_accuracy: 0.9671 - lr: 1.0000e-04
Epoch 4/20
1181/1181 [==============================] - 169s 143ms/step - loss: 0.0985 - accuracy: 0.9683 - val_loss: 0.1085 - val_accuracy: 0.9690 - lr: 1.0000e-04
Epoch 5/20
1181/1181 [==============================] - 157s 133ms/step - loss: 0.0958 - accuracy: 0.9699 - val_loss: 0.1035 - val_accuracy: 0.9695 - lr: 5.0000e-05
Epoch 6/20
1181/1181 [==============================] - 173s 146ms/step - loss: 0.0954 - accuracy: 0.9699 - val_loss: 0.1028 - val_accuracy: 0.9697 - lr: 5.0000e-05
Epoch 7/20

In [48]:
#Unfreezing second last layer of base_model 
base_model.layers[-2].trainable = True

In [49]:
#Compiling the model with a Adam optimizer with a relatively even lower Learning Rate
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
metrics=["accuracy"])

In [52]:
stage3_checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath='stage3_model_weights_epoch{epoch:02d}.h5', 
    save_weights_only=True,  
    save_freq='epoch')

In [53]:
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[lr_scheduler,early_stopping_cb,stage3_checkpoint_cb])

Epoch 1/20
1181/1181 [==============================] - 177s 146ms/step - loss: 0.0916 - accuracy: 0.9712 - val_loss: 0.1017 - val_accuracy: 0.9688 - lr: 1.0000e-05
Epoch 2/20
1181/1181 [==============================] - 157s 133ms/step - loss: 0.0915 - accuracy: 0.9709 - val_loss: 0.1009 - val_accuracy: 0.9690 - lr: 1.0000e-05
Epoch 3/20
1181/1181 [==============================] - 168s 142ms/step - loss: 0.0914 - accuracy: 0.9716 - val_loss: 0.1003 - val_accuracy: 0.9695 - lr: 1.0000e-05
Epoch 4/20
1181/1181 [==============================] - 171s 145ms/step - loss: 0.0913 - accuracy: 0.9713 - val_loss: 0.1003 - val_accuracy: 0.9690 - lr: 1.0000e-05
Epoch 5/20
1181/1181 [==============================] - 158s 133ms/step - loss: 0.0913 - accuracy: 0.9713 - val_loss: 0.0994 - val_accuracy: 0.9707 - lr: 1.0000e-05
Epoch 6/20
1181/1181 [==============================] - 168s 142ms/step - loss: 0.0913 - accuracy: 0.9712 - val_loss: 0.1004 - val_accuracy: 0.9690 - lr: 1.0000e-05
Epoch 7/20

In [29]:
model.load_weights('/kaggle/working/stage3_model_weights_epoch12.h5')

In [54]:
data_reshaped = data_val.values.reshape(-1, 28, 28, 1) / 255.0

In [58]:
def test_preprocess(image):
    rgb_image = tf.image.grayscale_to_rgb(tf.convert_to_tensor(image)) #Converting the Images to RGB from Grayscale
    resized_image = tf.image.resize(rgb_image, (224, 224)) #Resizing the Images for use in Resnet50
    final_image = keras.applications.resnet50.preprocess_input(resized_image)
    return final_image

In [59]:
batch_size = 32
num_samples = len(data_reshaped) 

all_predictions = []


for batch_start in range(0, num_samples, batch_size):
    batch_end = batch_start + batch_size
    batch_data = data_reshaped[batch_start:batch_end]  

    preprocessed_batch = test_preprocess(batch_data)

    batch_predictions = model.predict(preprocessed_batch)

    all_predictions.append(batch_predictions)

all_predictions = np.concatenate(all_predictions, axis=0)

1/1 [==============================] - 0s 36ms/step


In [60]:
predictions = all_predictions.tolist()

In [61]:
predictions = [x.index(max(x)) for x in predictions]

In [62]:
dict = {'ImageId': [x for x in range(1,len(data_val) + 1)] , 'Label': predictions}

In [63]:
submission = pd.DataFrame(dict)

In [64]:
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [65]:
submission.to_csv('mnist_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
